In [24]:
home_directory = "/Users/benjaminchang/Documents/poe/"

In [25]:
!pip install transformers

In [26]:
!pip install nltk

In [27]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/benjaminchang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/benjaminchang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Import Libraries & Load Data

In [28]:
import numpy as np
import pandas as pd 

import random
import time
import datetime

import matplotlib.pyplot as plt
plt.style.use('bmh')

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

pd.set_option('display.max_rows', 100)

In [29]:
poem_line_df = pd.read_csv('/Users/benjaminchang/Documents/poe/maryoliver_lines.csv')
poem_line_df = poem_line_df.fillna('')

# Text Generation - GPT-2
## Process Text and Create Dataset

In [30]:
RANDOM_SEED = 73
BATCH_SIZE = 4

EPOCHS = 5
SAMPLE_EVERY = 100

MAX_SEQUENCE_LENGTH = 512

In [31]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [32]:
class PoePoemDataset(Dataset):
    
    def __init__(self, data, tokenizer, gpt2_type='gpt2', max_length=MAX_SEQUENCE_LENGTH):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        
        for i in data:
            encodings_dict = tokenizer('<BOS>' + i + '<EOS>',
                                     truncation=True,
                                     max_length=max_length,
                                     padding='max_length'
                                    )

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        
        return self.input_ids[idx], self.attn_masks[idx]
        

In [33]:
poem_line_dataset = PoePoemDataset(poem_line_df['line_text'].values, tokenizer, max_length=MAX_SEQUENCE_LENGTH)

### Train/Validation Split

In [34]:
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size

In [35]:
poem_line_train_size, poem_line_val_size = train_val_split(0.8, poem_line_dataset)

# random split imported from troch.utils
poem_line_train_dataset, poem_line_val_dataset = random_split(poem_line_dataset, [poem_line_train_size, poem_line_val_size])

### Apply Random Seeds

In [36]:
# torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

### Instantiate DataLoaders and Define Model Creation Function

---



In [37]:
def create_dataloaders(train_dataset, val_dataset, bs):
    train_dataloader = DataLoader(train_dataset,
                                  sampler=RandomSampler(train_dataset),
                                  batch_size=bs)

    val_dataloader = DataLoader(val_dataset,
                                sampler=SequentialSampler(val_dataset),
                                batch_size=bs)
    
    return train_dataloader, val_dataloader

In [38]:
poem_line_train_dataloader, poem_line_val_dataloader = create_dataloaders(poem_line_train_dataset, poem_line_val_dataset, BATCH_SIZE)

In [39]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_SEQUENCE_LENGTH).from_pretrained('gpt2', output_hidden_states=True)

In [40]:
# helper function for logging time
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [41]:
# hyperparameters

learning_rate = 5e-4
eps = 1e-8
warmup_steps = 1e2

In [42]:
# create text generation seed prompt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
prompt = "<BOS>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

In [43]:
def create_model(train_dataloader, val_dataloader, file_name):

    model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
    model.resize_token_embeddings(len(tokenizer))
    
    # model.cuda()
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)

    total_steps = len(train_dataloader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=warmup_steps,
                                                num_training_steps=total_steps)
    
    total_t0 = time.time()
    model = model.to(device)

    for epoch_i in range(0, EPOCHS):

        print(f'Epoch {epoch_i + 1} of {EPOCHS}')

        t0 = time.time()
        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):

            print(f"Step: {step} in Epoch {epoch_i}")

            b_input_ids = batch[0].to(device)
            b_labels = batch[0].to(device)
            b_masks = batch[1].to(device)

            model.zero_grad()        

            outputs = model(b_input_ids,
                            labels=b_labels, 
                            attention_mask=b_masks,
                            token_type_ids=None)

            loss = outputs[0]  

            batch_loss = loss.item()
            total_train_loss += batch_loss

            if step % SAMPLE_EVERY == 0 and step != 0:
                
                model.eval()
                sample_outputs = model.generate(
                                        generated,
                                        do_sample=True,   
                                        top_k=50, 
                                        max_length=200,
                                        top_p=0.95, 
                                        num_return_sequences=1
                                    )
                for i, sample_output in enumerate(sample_outputs):
                      print(f'Example output: {tokenizer.decode(sample_output, skip_special_tokens=True)}')

                model.train()

            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)       
        training_time = format_time(time.time() - t0)

        print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

        t0 = time.time()

        print('Evaluating Model')

        model.eval()

        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in val_dataloader:
            b_input_ids = batch[0].to(device)
            b_labels = batch[0].to(device)
            b_masks = batch[1].to(device)

            with torch.no_grad():        

                outputs  = model(b_input_ids,  
                                 attention_mask=b_masks,
                                 labels=b_labels)

                loss = outputs[0]  

            batch_loss = loss.item()
            total_eval_loss += batch_loss        

        avg_val_loss = total_eval_loss / len(val_dataloader)

        validation_time = format_time(time.time() - t0)    

        print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')

    print(f'Total training took {format_time(time.time()-total_t0)}')

    torch.save(model.state_dict(), home_directory + file_name)
    return model

# Create Poem Line Model

In [44]:
poem_line_model = create_model(poem_line_train_dataloader, poem_line_val_dataloader, 'poem_line_model_4_epoch.pth')

/Users/benjaminchang/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch 1 of 5
Step: 0 in Epoch 0
Step: 1 in Epoch 0
Step: 2 in Epoch 0
Step: 3 in Epoch 0
Step: 4 in Epoch 0
Step: 5 in Epoch 0
Step: 6 in Epoch 0
Step: 7 in Epoch 0
Step: 8 in Epoch 0
Step: 9 in Epoch 0
Step: 10 in Epoch 0
Step: 11 in Epoch 0
Step: 12 in Epoch 0
Step: 13 in Epoch 0
Step: 14 in Epoch 0
Step: 15 in Epoch 0
Step: 16 in Epoch 0
Step: 17 in Epoch 0
Step: 18 in Epoch 0
Step: 19 in Epoch 0
Step: 20 in Epoch 0
Step: 21 in Epoch 0
Step: 22 in Epoch 0
Step: 23 in Epoch 0
Step: 24 in Epoch 0
Step: 25 in Epoch 0
Step: 26 in Epoch 0
Step: 27 in Epoch 0
Step: 28 in Epoch 0
Step: 29 in Epoch 0
Step: 30 in Epoch 0
Step: 31 in Epoch 0
Step: 32 in Epoch 0
Step: 33 in Epoch 0
Step: 34 in Epoch 0
Step: 35 in Epoch 0
Step: 36 in Epoch 0
Step: 37 in Epoch 0
Step: 38 in Epoch 0
Step: 39 in Epoch 0
Step: 40 in Epoch 0
Step: 41 in Epoch 0
Step: 42 in Epoch 0
Step: 43 in Epoch 0
Step: 44 in Epoch 0
Step: 45 in Epoch 0
Step: 46 in Epoch 0
Step: 47 in Epoch 0
Step: 48 in Epoch 0
Step: 49 in Epoch

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example output: which is that place of light and light, the day.
Step: 101 in Epoch 0
Step: 102 in Epoch 0
Step: 103 in Epoch 0
Step: 104 in Epoch 0
Step: 105 in Epoch 0
Step: 106 in Epoch 0
Step: 107 in Epoch 0
Step: 108 in Epoch 0
Step: 109 in Epoch 0
Step: 110 in Epoch 0
Step: 111 in Epoch 0
Step: 112 in Epoch 0
Step: 113 in Epoch 0
Step: 114 in Epoch 0
Step: 115 in Epoch 0
Step: 116 in Epoch 0
Step: 117 in Epoch 0
Step: 118 in Epoch 0
Step: 119 in Epoch 0
Step: 120 in Epoch 0
Step: 121 in Epoch 0
Step: 122 in Epoch 0
Step: 123 in Epoch 0
Step: 124 in Epoch 0
Step: 125 in Epoch 0
Step: 126 in Epoch 0
Step: 127 in Epoch 0
Step: 128 in Epoch 0
Step: 129 in Epoch 0
Step: 130 in Epoch 0
Step: 131 in Epoch 0
Step: 132 in Epoch 0
Step: 133 in Epoch 0
Step: 134 in Epoch 0
Step: 135 in Epoch 0
Step: 136 in Epoch 0
Step: 137 in Epoch 0
Step: 138 in Epoch 0
Step: 139 in Epoch 0
Step: 140 in Epoch 0
Step: 141 in Epoch 0
Step: 142 in Epoch 0
Step: 143 in Epoch 0
Step: 144 in Epoch 0
Step: 145 i

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Example output: the mountains and the deep trees.
Step: 301 in Epoch 4
Step: 302 in Epoch 4
Step: 303 in Epoch 4
Step: 304 in Epoch 4
Step: 305 in Epoch 4
Step: 306 in Epoch 4
Step: 307 in Epoch 4
Step: 308 in Epoch 4
Step: 309 in Epoch 4
Step: 310 in Epoch 4
Step: 311 in Epoch 4
Step: 312 in Epoch 4
Step: 313 in Epoch 4
Step: 314 in Epoch 4
Step: 315 in Epoch 4
Step: 316 in Epoch 4
Step: 317 in Epoch 4
Step: 318 in Epoch 4
Step: 319 in Epoch 4
Step: 320 in Epoch 4
Step: 321 in Epoch 4
Step: 322 in Epoch 4
Step: 323 in Epoch 4
Step: 324 in Epoch 4
Step: 325 in Epoch 4
Step: 326 in Epoch 4
Step: 327 in Epoch 4
Step: 328 in Epoch 4
Step: 329 in Epoch 4
Step: 330 in Epoch 4
Step: 331 in Epoch 4
Step: 332 in Epoch 4
Step: 333 in Epoch 4
Step: 334 in Epoch 4
Step: 335 in Epoch 4
Step: 336 in Epoch 4
Step: 337 in Epoch 4
Step: 338 in Epoch 4
Step: 339 in Epoch 4
Step: 340 in Epoch 4
Step: 341 in Epoch 4
Step: 342 in Epoch 4
Step: 343 in Epoch 4
Step: 344 in Epoch 4
Step: 345 in Epoch 4
Step:

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example output: and there’s one’s in me that’s perfectly content—
Step: 401 in Epoch 4
Step: 402 in Epoch 4
Step: 403 in Epoch 4
Step: 404 in Epoch 4
Step: 405 in Epoch 4
Step: 406 in Epoch 4
Step: 407 in Epoch 4
Step: 408 in Epoch 4
Step: 409 in Epoch 4
Step: 410 in Epoch 4
Step: 411 in Epoch 4
Step: 412 in Epoch 4
Step: 413 in Epoch 4
Step: 414 in Epoch 4
Step: 415 in Epoch 4
Step: 416 in Epoch 4
Step: 417 in Epoch 4
Step: 418 in Epoch 4
Step: 419 in Epoch 4
Step: 420 in Epoch 4
Step: 421 in Epoch 4
Step: 422 in Epoch 4
Step: 423 in Epoch 4
Step: 424 in Epoch 4
Step: 425 in Epoch 4
Step: 426 in Epoch 4
Step: 427 in Epoch 4
Step: 428 in Epoch 4
Step: 429 in Epoch 4
Step: 430 in Epoch 4
Step: 431 in Epoch 4
Step: 432 in Epoch 4
Step: 433 in Epoch 4
Step: 434 in Epoch 4
Step: 435 in Epoch 4
Step: 436 in Epoch 4
Step: 437 in Epoch 4
Step: 438 in Epoch 4
Step: 439 in Epoch 4
Step: 440 in Epoch 4
Step: 441 in Epoch 4
Step: 442 in Epoch 4
Step: 443 in Epoch 4
Step: 444 in Epoch 4
Step: 445 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example output: He carries a silver leaf, and is gone.
Step: 501 in Epoch 4
Step: 502 in Epoch 4
Step: 503 in Epoch 4
Step: 504 in Epoch 4
Step: 505 in Epoch 4
Step: 506 in Epoch 4
Step: 507 in Epoch 4
Step: 508 in Epoch 4
Step: 509 in Epoch 4
Step: 510 in Epoch 4
Step: 511 in Epoch 4
Step: 512 in Epoch 4
Step: 513 in Epoch 4
Step: 514 in Epoch 4
Step: 515 in Epoch 4
Step: 516 in Epoch 4
Step: 517 in Epoch 4
Step: 518 in Epoch 4
Step: 519 in Epoch 4
Step: 520 in Epoch 4
Step: 521 in Epoch 4
Step: 522 in Epoch 4
Step: 523 in Epoch 4
Step: 524 in Epoch 4
Step: 525 in Epoch 4
Step: 526 in Epoch 4
Step: 527 in Epoch 4
Step: 528 in Epoch 4
Step: 529 in Epoch 4
Step: 530 in Epoch 4
Step: 531 in Epoch 4
Step: 532 in Epoch 4
Step: 533 in Epoch 4
Step: 534 in Epoch 4
Step: 535 in Epoch 4
Step: 536 in Epoch 4
Step: 537 in Epoch 4
Step: 538 in Epoch 4
Step: 539 in Epoch 4
Step: 540 in Epoch 4
Step: 541 in Epoch 4
Step: 542 in Epoch 4
Step: 543 in Epoch 4
Step: 544 in Epoch 4
Step: 545 in Epoch 4


# Generate Poem Lines

In [47]:
poem_line_model.eval()

sample_outputs = poem_line_model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length=512,
                                top_p=0.95, 
                                num_return_sequences=30
                                )

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: to see how the world changes.


1: I don’t hear him all afternoon,


2: with which you perceive


3: And the old gods liked so well.


4: You only have to let the soft animal of your body


5: It grew under the tiger’s orange paws.


6: It has lasted to this day.


7: I don’t suppose the bird might too casual be


8: as one of them, as though it was your own vanishing song


9: to his own madness


10: the birds in the trees, especially those with golden wings.


11: of my blood,


12: and then it hurried


13: that is beautiful.


14: or she’d hang in the milky moonlight


15: a few drops of dark seed seed had gone green,


16: and never once mentioned the word name.


17: the blue-smocked lilies of Monet.


18: where to climb, and where to climb


19: and I hope I wouldn’t.


20: but a few raisins give him a little more breathing.


21: and opened his eyes.


22: in the face of adversity.


23: the perfect commotion of flowers and the perfect petals


24: where the world is full o